# MBA

In [3]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
import mlxtend as ml
import sqlite3

### Querry Data

In [4]:
conn = sqlite3.connect("data/liquor.db")
query = '''
SELECT Date, t."Store Number", "Store Type", p.product_name, category_new, "Volume Sold", County
FROM transactions AS t
LEFT JOIN stores AS s ON t."Store Number" = s."Store Number"
LEFT JOIN products AS p ON t."Item Number" = p."Item Number"
'''
df = pd.read_sql_query(query, conn)
conn.close()
df.head(3)

,Date,Store Number,Store Type,product_name,category_new,Volume Sold,County
0,2012-05-03 00:00:00,2636,Liquor Tobacco Store,nikolai vodka,vodka,21.0,polk
1,2012-07-05 00:00:00,3735,Liquor Tobacco Store,kessler blend whiskey,whisky,1.2,buchanan
2,2012-03-06 00:00:00,4631,Other,uv cake vodka,vodka,9.0,black hawk


In [4]:
df.shape

(19445831, 7)

In [170]:
df.dtypes

Date            datetime64[ns]
Store Number            object
Store Type              object
product_name            object
category_new            object
Volume Sold            float64
County                  object
Month                    int64
dtype: object

In [10]:
# Make Date column datetime
df.loc[:,'Date']=pd.to_datetime(df.Date)


### Subset df by filtering only most popular items and longest running stores

In [95]:
# Most Popular Items
# Want the top 200 # 9587 Items Total
top200 = df.groupby(['product_name']
                   ).agg({'Volume Sold': 'sum'}
                        ).sort_values(by= 'Volume Sold', 
                                      ascending= False
                                     ).reset_index()[0:200]['product_name'].tolist()

# Get longevity
long_g = df.groupby(['Store Number']
                   ).agg({'Date': (lambda x: (max(x)-min(x)).days)}
                        ).sort_values(by='Date',ascending=False
                                     ).reset_index()
# 3129 Days is the 75% longevity percentile (3163 max). 
# 614 stores out of 2448 stores total will be included in MBA
long_store = long_g.loc[long_g.Date >= 3129]['Store Number'].tolist()

# Filter the popular items and longest running stores
# 5443606 rows × 7 columns
df = df.loc[df['Store Number'].isin(long_store)]
df = df.loc[df['product_name'].isin(top200)]
df.head(3)

,Date,Store Number,Store Type,product_name,category_new,Volume Sold,County
0,2012-05-03,2636,Liquor Tobacco Store,nikolai vodka,vodka,21.0,polk
5,2013-08-05,3833,Supermarket,jagermeister liqueur,liqueur,18.0,cass
7,2014-04-28,4626,Convenience Store,paramount white rum,rum,9.0,pottawatta


In [89]:
df.shape

(7952034, 7)

## Make data for MBA R

In [121]:
## get rid of commas and quotes in product names to avoid complications
df.loc[:,'product_name'] = df.product_name.replace(',','', regex=True)
df.loc[:,'product_name'] = df.product_name.replace("'",'', regex=True)

In [129]:
df.head(1)

,Date,Store Number,Store Type,product_name,category_new,Volume Sold,County
0,2012-05-03,2636,Liquor Tobacco Store,nikolai vodka,vodka,21.0,polk


### General mba for R

In [124]:
## Groupby and make list of product names for each group
df_gen = df.groupby([pd.Grouper(key='Date', freq='M'),
                     'Store Number']).agg({'product_name':lambda x: list(np.unique(x))}).reset_index()

## Turn list of product names into columns
df_gen[['v' + i for i in map(str, range(1,176))]]= pd.DataFrame(df_gen['product_name'].values.tolist(), index = df_r.index)

## Write to csv for r
#df_r.drop(['product_name','Store Number', 'Date'], axis=1).to_csv('data/mba_test.csv', index=False)

# Should be 63039 rows and 178 columns
df_r.shape

### Supermarket mba for R

In [132]:
df_groc = df.loc[df['Store Type']=='Supermarket']
df_groc.shape

(4094216, 7)

In [136]:
## Groupby and make list of product names for each group
df_groc = df.loc[df['Store Type']=='Supermarket']

df_groc = df_groc.groupby([pd.Grouper(key='Date', freq='M'),
                     'Store Number']).agg({'product_name':lambda x: list(np.unique(x))}).reset_index()

## Turn list of product names into columns
df_groc[['v' + i for i in map(str, range(1,176))]]= pd.DataFrame(df_groc['product_name'].values.tolist(), index = df_groc.index)

## Write to csv for r
#df_groc.drop(['product_name','Store Number', 'Date'], axis=1).to_csv('data/mba_groc.csv', index=False)


df_groc.shape

(27531, 178)

### Liquor and Tobacco Store MBA for R

In [143]:
## Groupby and make list of product names for each group
df_liq = df.loc[df['Store Type']=='Liquor Tobacco Store']

df_liq = df_liq.groupby([pd.Grouper(key='Date', freq='M'),
                     'Store Number']).agg({'product_name':lambda x: list(np.unique(x))}).reset_index()

## Turn list of product names into columns
df_liq[['v' + i for i in map(str, range(1,173))]]= pd.DataFrame(df_liq['product_name'].values.tolist(), index = df_liq.index)

## Write to csv for r
#df_liq.drop(['product_name','Store Number', 'Date'], axis=1).to_csv('data/mba_liq.csv', index=False)


df_liq.shape

(9929, 175)

### Convenience Store MBA for R

In [148]:
## Groupby and make list of product names for each group
df_con = df.loc[df['Store Type']=='Convenience Store']

df_con = df_con.groupby([pd.Grouper(key='Date', freq='M'),
                     'Store Number']).agg({'product_name':lambda x: list(np.unique(x))}).reset_index()

## Turn list of product names into columns
df_con[['v' + i for i in map(str, range(1,144))]]= pd.DataFrame(df_con['product_name'].values.tolist(), index = df_con.index)

## Write to csv for r
#df_con.drop(['product_name','Store Number', 'Date'], axis=1).to_csv('data/mba_con.csv', index=False)


df_con.shape

(15785, 146)

In [137]:
df['Store Type'].unique()

array(['Liquor Tobacco Store', 'Supermarket', 'Convenience Store',
       'Drug Store', 'Other Grocery or Convenience', 'Other', 'Casino'],
      dtype=object)

### Drug Stores MBA for R

In [152]:
## Groupby and make list of product names for each group
df_drug = df.loc[df['Store Type']=='Drug Store']

df_drug = df_drug.groupby([pd.Grouper(key='Date', freq='M'),
                     'Store Number']).agg({'product_name':lambda x: list(np.unique(x))}).reset_index()

## Turn list of product names into columns
df_drug[['v' + i for i in map(str, range(1,169))]]= pd.DataFrame(df_drug['product_name'].values.tolist(), index = df_drug.index)

## Write to csv for r
#df_drug.drop(['product_name','Store Number', 'Date'], axis=1).to_csv('data/mba_drug.csv', index=False)


df_drug.shape

(3280, 171)

# Seasons

In [153]:
df['Month'] = pd.DatetimeIndex(df.Date).month

In [155]:
df.head(1)

,Date,Store Number,Store Type,product_name,category_new,Volume Sold,County,Month
0,2012-05-03,2636,Liquor Tobacco Store,nikolai vodka,vodka,21.0,polk,5


### SPRING MBA for R

In [158]:
df_sp = df.loc[df['Month'].isin([3,4,5])]
df_sp = df_sp.groupby([pd.Grouper(key='Date', freq='M'),
                       'Store Number']).agg({'product_name':lambda x: list(np.unique(x))}).reset_index()

## Turn list of product names into columns
df_sp[['v' + i for i in map(str, range(1,176))]]= pd.DataFrame(df_sp['product_name'].values.tolist(), index = df_sp.index)

## Write to csv for r
# df_sp.drop(['product_name','Store Number', 'Date'], axis=1).to_csv('data/mba_sp.csv', index=False)


df_sp.shape

(16354, 178)

### SUMMER MBA for R

In [161]:
df_su = df.loc[df['Month'].isin([6,7,8])]
df_su = df_su.groupby([pd.Grouper(key='Date', freq='M'),
                       'Store Number']).agg({'product_name':lambda x: list(np.unique(x))}).reset_index()

## Turn list of product names into columns
df_su[['v' + i for i in map(str, range(1,174))]]= pd.DataFrame(df_su['product_name'].values.tolist(), index = df_su.index)

## Write to csv for r
df_su.drop(['product_name','Store Number', 'Date'], axis=1).to_csv('data/mba_su.csv', index=False)


df_su.shape

(16380, 176)

### FALL MBA for R

In [164]:
df_fa = df.loc[df['Month'].isin([9,10,11])]
df_fa = df_fa.groupby([pd.Grouper(key='Date', freq='M'),
                       'Store Number']).agg({'product_name':lambda x: list(np.unique(x))}).reset_index()

## Turn list of product names into columns
df_fa[['v' + i for i in map(str, range(1,173))]]= pd.DataFrame(df_fa['product_name'].values.tolist(), index = df_fa.index)

## Write to csv for r
df_fa.drop(['product_name','Store Number', 'Date'], axis=1).to_csv('data/mba_fa.csv', index=False)


df_fa.shape

(14545, 175)

### WINTER MAB for R

In [167]:
df_wi = df.loc[df['Month'].isin([12,1,2])]
df_wi = df_wi.groupby([pd.Grouper(key='Date', freq='M'),
                       'Store Number']).agg({'product_name':lambda x: list(np.unique(x))}).reset_index()

## Turn list of product names into columns
df_wi[['v' + i for i in map(str, range(1,175))]]= pd.DataFrame(df_wi['product_name'].values.tolist(), index = df_wi.index)

## Write to csv for r
df_wi.drop(['product_name','Store Number', 'Date'], axis=1).to_csv('data/mba_wi.csv', index=False)


df_wi.shape

(15760, 177)

## POPULATION

In [166]:
#df_wi.groupby([pd.Grouper(key='Date', freq='M'),
                     'Store Number']).agg({'product_name':lambda x: len(list(np.unique(x)))}).max()



product_name    174
dtype: int64

# PYTHON Overall MBA

In [18]:
df_all = df[['Date', 'Store Number', 'product_name']].groupby([pd.Grouper(key='Date',
                                freq='M'
                               ),
                     'Store Number',
                     'product_name']
                   ).size().reset_index(name='count')

In [55]:
df_all['count'].max()
# 37

37

In [94]:
df.shape

(7952034, 7)

In [6]:
basket = (df_all.groupby([pd.Grouper(key='Date', freq='M'),'Store Number','product_name'])['count'].sum().unstack().reset_index().fillna(0)).set_index(['Date','Store Number'])#The encoding function

In [105]:
basket.head()

product_name             absolut citron (lemon vodka)  \
Date       Store Number                                 
2012-01-31 2106                                   1.0   
           2130                                   1.0   
           2178                                   1.0   
           2190                                  16.0   
           2191                                   3.0   

product_name             absolut swedish vodka 80 prf  \
Date       Store Number                                 
2012-01-31 2106                                   3.0   
           2130                                   4.0   
           2178                                   0.0   
           2190                                  23.0   
           2191                                   4.0   

product_name             admiral nelson spiced rum  \
Date       Store Number                              
2012-01-31 2106                                4.0   
           2130                                1.0   
           2178                                0.0   
           2190                                7.0   
           2191                                3.0   

product_name             admiral nelson spiced rum traveler  \
Date       Store Number                                       
2012-01-31 2106                                         0.0   
           2130                                         0.0   
           2178                                         0.0   
           2190                                         0.0   
           2191                                         0.0   

product_name             ancient age bourbon  arrow peppermint schnapps  \
Date       Store Number                                                   
2012-01-31 2106                          0.0                        1.0   
           2130                          0.0                        4.0   
           2178                          1.0                        0.0   
           2190                          2.0                        3.0   
           2191                          3.0                        9.0   

product_name             bacardi dragon berry  bacardi gold rum  \
Date       Store Number                                           
2012-01-31 2106                           1.0               0.0   
           2130                           0.0               0.0   
           2178                           1.0               0.0   
           2190                           9.0               9.0   
           2191                           1.0               2.0   

product_name             bacardi limon  bacardi superior pet  \
Date       Store Number                                        
2012-01-31 2106                    4.0                   0.0   
           2130                    3.0                   0.0   
           2178                    0.0                   0.0   
           2190                   16.0                   0.0   
           2191                    1.0                   0.0   

product_name             bacardi superior rum  bacardi superior rum pet  \
Date       Store Number                                                   
2012-01-31 2106                           8.0                       0.0   
           2130                           4.0                       0.0   
           2178                           0.0                       0.0   
           2190                          21.0                       0.0   
           2191                           3.0                       0.0   

product_name             bailey's original irish cream  barton gin  \
Date       Store Number                                              
2012-01-31 2106                                    3.0         2.0   
           2130                                    3.0         1.0   
           2178                                    0.0         0.0   
           2190                                   17.0    

In [7]:
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1
basket_sets = basket.applymap(encode_units)

In [29]:
frequent_itemsets = apriori(basket_sets, min_support=0.5, use_colnames=True, max_len=2)

In [112]:
frequent_itemsets.head()

,support,itemsets
0,0.708292,(absolut swedish vodka 80 prf)
1,0.700455,(admiral nelson spiced rum)
2,0.412380,(bacardi limon)
3,0.710417,(bacardi superior rum)
4,0.439522,(bailey's original irish cream)


In [30]:
rules = association_rules(frequent_itemsets, metric="lift")
# rules.sort_values('confidence', ascending = False, inplace = True)
# rules.head(10)

In [31]:
rules.sort_values('confidence', ascending = False, inplace = True)
rules.shape

(260, 9)

- Note: Hawkeye brand made in Missouri marketed towards residents of Iowa, which is known as "the Hawkeye State". Products sold under the Hawkeye name include blended whiskey, rum, gin, flavored vodka, and the most popular product – vodka
- Black Velvet (whiskey) is the most popular alcohol in iowa


In [22]:
rules.loc[(rules.confidence >.90)&(rules.support >.55)].sort_values('lift', ascending = False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
204,(paramount white rum),(hawkeye vodka),0.638763,0.850981,0.596916,0.934487,1.098129,0.053341,2.274656
40,(admiral nelson spiced rum),(hawkeye vodka),0.700455,0.850981,0.650153,0.928186,1.090725,0.054079,2.075079
203,(malibu coconut rum),(hawkeye vodka),0.639826,0.850981,0.587446,0.918134,1.078912,0.042966,1.820269
215,(uv blue (raspberry) vodka),(hawkeye vodka),0.647187,0.850981,0.589048,0.910167,1.069550,0.038304,1.658846
200,(jose cuervo especial reposado tequila),(hawkeye vodka),0.621504,0.850981,0.564698,0.908599,1.067708,0.035810,1.630385
209,(smirnoff vodka 80 prf),(hawkeye vodka),0.666857,0.850981,0.601548,0.902065,1.060029,0.034065,1.521607
12,(absolut swedish vodka 80 prf),(hawkeye vodka),0.708292,0.850981,0.638446,0.901389,1.059234,0.035703,1.511172
64,(bacardi superior rum),(hawkeye vodka),0.710417,0.850981,0.640254,0.901237,1.059056,0.035703,1.508854
167,(fireball cinnamon whiskey),(hawkeye vodka),0.685703,0.850981,0.617871,0.901078,1.058870,0.034352,1.506429
196,(jagermeister liqueur),(hawkeye vodka),0.613144,0.850981,0.551992,0.900264,1.057913,0.030217,1.494132


## Store Type

In [139]:
def mba_org(store_t, df1=df, min_s=0.5, max_l=2):
    df = df1.loc[df1['Store Type']==store_t].groupby([pd.Grouper(key='Date',freq='M'),'Store Number','product_name']).size().reset_index(name='count')

    basket = (df.groupby([pd.Grouper(key='Date', 
                                               freq='M'
                                              ),
                                    'Store Number',
                                    'product_name'
                                   ])['count'].sum().unstack().reset_index().fillna(0)).set_index(['Date',
                                                                                                   'Store Number'])

    basket_sets = basket.applymap(encode_units)

    frequent_itemsets = apriori(basket_sets, min_support=min_s, use_colnames=True, max_len=max_l)

    rules = association_rules(frequent_itemsets, metric="lift")
    
    return rules, frequent_itemsets

#### Define function to filter for store type and season and run mba

In [166]:
def mba(store_t=None, season=None, df1=df, min_s=0.5, max_l=2):
    if store_t == None:
        df = df1.loc[df1['Month'].isin(season)
                        ].groupby([pd.Grouper(key='Date',
                                              freq='M'
                                             ),
                                   'Store Number',
                                   'product_name'
                                  ]).size().reset_index(name='count')
        print('Season')
    elif season == None:
        df = df1.loc[df1['Store Type']==store_t
                    ].groupby([pd.Grouper(key='Date',
                                          freq='M'
                                         ),
                               'Store Number',
                               'product_name'
                              ]).size().reset_index(name='count')
        print('Store Type')
    else:
        return None

    basket = (df.groupby([pd.Grouper(key='Date', 
                                               freq='M'
                                              ),
                                    'Store Number',
                                    'product_name'
                                   ])['count'].sum().unstack().reset_index().fillna(0)).set_index(['Date',
                                                                                                   'Store Number'])

    basket_sets = basket.applymap(encode_units)

    frequent_itemsets = apriori(basket_sets, min_support=min_s, use_colnames=True, max_len=max_l)

    rules = association_rules(frequent_itemsets, metric="lift")
    
    return rules, frequent_itemsets


In [68]:
df.loc[df['Store Type']=='Supermarket'].shape

(4094216, 7)

In [73]:
df.loc[df['Store Type']=='Liquor Tobacco Store'].shape

(2030640, 7)

In [67]:
df.loc[df['Store Type']=='Convenience Store'].shape

(778505, 7)

In [71]:
df.loc[df['Store Type']=='Drug Store'].shape

(535619, 7)

In [69]:
df.loc[df['Store Type']=='Casino'].shape

(16073, 7)

#### SUPERMARKETS

In [168]:
rules_groc, frequent_itemsets_groc = mba(store_t='Supermarket')
rules_groc.loc[(rules_groc.confidence >.90)&(rules_groc.support >.55)].sort_values('lift', ascending = False).head(20)

Store Type


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
429,(jagermeister liqueur),(seagrams 7 crown bl whiskey),0.604863,0.855349,0.559048,0.924256,1.080561,0.041679,1.909739
511,(ten high),(seagrams 7 crown bl whiskey),0.605790,0.855349,0.556057,0.917905,1.073136,0.037896,1.761999
459,(jose cuervo authentic lime margarita),(seagrams 7 crown bl whiskey),0.665421,0.855349,0.607943,0.913622,1.068128,0.038776,1.674630
501,(phillips vodka),(seagrams 7 crown bl whiskey),0.616406,0.855349,0.562248,0.912139,1.066394,0.035006,1.646359
469,(jose cuervo especial reposado tequila),(seagrams 7 crown bl whiskey),0.682885,0.855349,0.622118,0.911014,1.065079,0.038013,1.625549
491,(paramount white rum),(seagrams 7 crown bl whiskey),0.765394,0.855349,0.694967,0.907986,1.061539,0.040289,1.572063
515,(uv blue (raspberry) vodka),(seagrams 7 crown bl whiskey),0.738180,0.855349,0.670176,0.907876,1.061410,0.038774,1.570174
341,(five star),(seagrams 7 crown bl whiskey),0.634708,0.855349,0.576004,0.907510,1.060983,0.033108,1.563975
477,(malibu coconut rum),(seagrams 7 crown bl whiskey),0.747271,0.855349,0.676486,0.905275,1.058369,0.037308,1.527060
77,(admiral nelson spiced rum),(seagrams 7 crown bl whiskey),0.779389,0.855349,0.703669,0.902847,1.055531,0.037020,1.488905


In [149]:
frequent_itemsets_groc.sort_values(by='support', ascending=False).head(20)

,support,itemsets
4,0.933610,(black velvet)
17,0.916594,(hawkeye vodka)
120,0.867490,"(black velvet, hawkeye vodka)"
33,0.855349,(seagrams 7 crown bl whiskey)
14,0.829810,(five o'clock vodka)
136,0.814199,"(seagrams 7 crown bl whiskey, black velvet)"
232,0.804031,"(seagrams 7 crown bl whiskey, hawkeye vodka)"
117,0.788092,"(black velvet, five o'clock vodka)"
1,0.779389,(admiral nelson spiced rum)
18,0.778462,(jack daniels old #7 black lbl)


#### LIQUOR STORES

In [150]:
rules_liq, frequent_itemsets_liq = mba(store_t='Liquor Tobacco Store')
rules_liq.loc[(rules_liq.confidence >.90)&(rules_liq.support >.55)].sort_values('lift', ascending = False).head(20)

Season


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
267,(dr. mcgillicuddy's cherry schnapps),(jack daniels old #7 black lbl),0.613778,0.801633,0.554859,0.904006,1.127705,0.062834,2.066448
411,(malibu coconut rum),(jack daniels old #7 black lbl),0.680366,0.801633,0.613417,0.901598,1.124702,0.068013,2.015883
307,(five star),(five o'clock vodka),0.616710,0.820716,0.563972,0.914484,1.114251,0.057828,2.096498
182,(dr. mcgillicuddy's cherry schnapps),(captain morgan spiced rum),0.613778,0.838536,0.570874,0.930099,1.109193,0.056199,2.309896
210,(rumchata),(captain morgan spiced rum),0.596725,0.838536,0.554633,0.929462,1.108434,0.054258,2.289044
205,(malibu coconut rum),(captain morgan spiced rum),0.680366,0.838536,0.630109,0.926132,1.104463,0.059597,2.185845
221,(uv blue (raspberry) vodka),(captain morgan spiced rum),0.662321,0.838536,0.610485,0.921736,1.099219,0.055104,2.063051
217,(southern comfort),(captain morgan spiced rum),0.678336,0.838536,0.625237,0.921721,1.099202,0.056427,2.062672
60,(admiral nelson spiced rum),(hawkeye vodka),0.668231,0.839800,0.615582,0.921213,1.096943,0.054403,2.033322
180,(crown royal canadian whisky),(captain morgan spiced rum),0.765316,0.838536,0.703510,0.919241,1.096244,0.061764,1.999319


In [151]:
frequent_itemsets_liq.sort_values(by='support', ascending=False).head(20)

,support,itemsets
5,0.916358,(black velvet)
19,0.839800,(hawkeye vodka)
9,0.838536,(captain morgan spiced rum)
16,0.820716,(five o'clock vodka)
102,0.805332,"(black velvet, captain morgan spiced rum)"
112,0.804566,"(black velvet, hawkeye vodka)"
22,0.801633,(jack daniels old #7 black lbl)
109,0.783587,"(five o'clock vodka, black velvet)"
113,0.771677,"(jack daniels old #7 black lbl, black velvet)"
34,0.768564,(seagrams 7 crown bl whiskey)


#### CONVENIENCE STORES

In [171]:
rules_conv, frequent_itemsets_conv = mba(store_t='Convenience Store')
rules_conv

Store Type


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(black velvet),(admiral nelson spiced rum),0.904569,0.538048,0.510445,0.564297,1.048786,0.023744,1.060246
1,(admiral nelson spiced rum),(black velvet),0.538048,0.904569,0.510445,0.948699,1.048786,0.023744,1.860233
2,(black velvet),(captain morgan spiced rum),0.904569,0.615677,0.571931,0.632269,1.026949,0.015009,1.045120
3,(captain morgan spiced rum),(black velvet),0.615677,0.904569,0.571931,0.928946,1.026949,0.015009,1.343087
4,(black velvet),(hawkeye vodka),0.904569,0.660406,0.616394,0.681422,1.031824,0.019011,1.065971
5,(hawkeye vodka),(black velvet),0.660406,0.904569,0.616394,0.933356,1.031824,0.019011,1.431953


In [153]:
frequent_itemsets_conv.sort_values(by='support', ascending=False).head(20)

,support,itemsets
1,0.904569,(black velvet)
5,0.660406,(hawkeye vodka)
9,0.616394,"(black velvet, hawkeye vodka)"
2,0.615677,(captain morgan spiced rum)
8,0.571931,"(black velvet, captain morgan spiced rum)"
0,0.538048,(admiral nelson spiced rum)
7,0.510445,"(black velvet, admiral nelson spiced rum)"
3,0.507253,(fireball cinnamon whiskey)
4,0.505964,(fireball cinnamon whiskey mini dispenser)
6,0.503570,(jim beam)


#### DRUG STORES

In [147]:
rules_drug, frequent_itemsets_drug = mba(store_t='Drug Store')
rules_drug_org.sort_values('lift', ascending = False)

Season


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
6,(black velvet),(smirnoff vodka 80 prf),0.774796,0.611226,0.505357,0.652246,1.067111,0.031782,1.117956
7,(smirnoff vodka 80 prf),(black velvet),0.611226,0.774796,0.505357,0.826793,1.067111,0.031782,1.300203
2,(admiral nelson spiced rum),(hawkeye vodka),0.544972,0.907402,0.514892,0.944805,1.041220,0.020384,1.677659
3,(hawkeye vodka),(admiral nelson spiced rum),0.907402,0.544972,0.514892,0.567436,1.041220,0.020384,1.051932
11,(hawkeye vodka),(jack daniels old #7 black lbl),0.907402,0.546446,0.515777,0.568411,1.040195,0.019930,1.050892
10,(jack daniels old #7 black lbl),(hawkeye vodka),0.546446,0.907402,0.515777,0.943875,1.040195,0.019930,1.649848
12,(paramount white rum),(hawkeye vodka),0.536125,0.907402,0.505554,0.942978,1.039206,0.019073,1.623890
13,(hawkeye vodka),(paramount white rum),0.907402,0.536125,0.505554,0.557144,1.039206,0.019073,1.047463
17,(smirnoff vodka 80 prf),(hawkeye vodka),0.611226,0.907402,0.575936,0.942264,1.038420,0.021309,1.603829
16,(hawkeye vodka),(smirnoff vodka 80 prf),0.907402,0.611226,0.575936,0.634709,1.038420,0.021309,1.064287


In [110]:
frequent_itemsets_drug.sort_values(by='support', ascending=False).head(20)

,support,itemsets
4,0.929878,(black velvet)
21,0.922866,(hawkeye vodka)
180,0.873171,"(black velvet, hawkeye vodka)"
34,0.872561,(mccormick vodka pet)
46,0.861280,(seagrams 7 crown bl whiskey)
328,0.822866,"(mccormick vodka pet, hawkeye vodka)"
192,0.821646,"(black velvet, mccormick vodka pet)"
201,0.821037,"(seagrams 7 crown bl whiskey, black velvet)"
337,0.812805,"(seagrams 7 crown bl whiskey, hawkeye vodka)"
1,0.801829,(admiral nelson spiced rum)


#### CASINOS

In [167]:
rules_cas, frequent_itemsets_cas = mba('Casino')
rules_cas.head(20)
# Not many casinos or data points

Store Type


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(absolut swedish vodka 80 prf),(barton vodka),0.682927,0.656504,0.510163,0.747024,1.137881,0.061818,1.357819
1,(barton vodka),(absolut swedish vodka 80 prf),0.656504,0.682927,0.510163,0.777090,1.137881,0.061818,1.422425
2,(absolut swedish vodka 80 prf),(captain morgan spiced rum),0.682927,0.808943,0.573171,0.839286,1.037509,0.020722,1.188799
3,(captain morgan spiced rum),(absolut swedish vodka 80 prf),0.808943,0.682927,0.573171,0.708543,1.037509,0.020722,1.087889
4,(crown royal canadian whisky),(absolut swedish vodka 80 prf),0.839431,0.682927,0.591463,0.704600,1.031736,0.018194,1.073371
5,(absolut swedish vodka 80 prf),(crown royal canadian whisky),0.682927,0.839431,0.591463,0.866071,1.031736,0.018194,1.198916
6,(grey goose vodka),(absolut swedish vodka 80 prf),0.782520,0.682927,0.556911,0.711688,1.042115,0.022506,1.099758
7,(absolut swedish vodka 80 prf),(grey goose vodka),0.682927,0.782520,0.556911,0.815476,1.042115,0.022506,1.178600
8,(jack daniels old #7 black lbl),(absolut swedish vodka 80 prf),0.819106,0.682927,0.577236,0.704715,1.031904,0.017847,1.073786
9,(absolut swedish vodka 80 prf),(jack daniels old #7 black lbl),0.682927,0.819106,0.577236,0.845238,1.031904,0.017847,1.168856


In [155]:
frequent_itemsets_cas.sort_values(by='support', ascending=False).head(20)

,support,itemsets
6,0.839431,(crown royal canadian whisky)
8,0.819106,(jack daniels old #7 black lbl)
5,0.808943,(captain morgan spiced rum)
7,0.782520,(grey goose vodka)
39,0.723577,"(jack daniels old #7 black lbl, crown royal ca..."
44,0.707317,"(grey goose vodka, jack daniels old #7 black lbl)"
34,0.697154,"(jack daniels old #7 black lbl, captain morgan..."
32,0.693089,"(crown royal canadian whisky, captain morgan s..."
38,0.689024,"(grey goose vodka, crown royal canadian whisky)"
0,0.682927,(absolut swedish vodka 80 prf)


## SEASONS

In [113]:
df['Month'] = pd.DatetimeIndex(df.Date).month

,Date,Store Number,Store Type,product_name,category_new,Volume Sold,County
0,2012-05-03 00:00:00,2636,Liquor Tobacco Store,nikolai vodka,vodka,21.0,polk
1,2012-07-05 00:00:00,3735,Liquor Tobacco Store,kessler blend whiskey,whisky,1.2,buchanan
2,2012-03-06 00:00:00,4631,Other,uv cake vodka,vodka,9.0,black hawk
3,2013-08-26 00:00:00,3162,Other Grocery or Convenience,windsor canadian pet,whisky,10.5,muscatine
4,2015-08-12 00:00:00,2513,Supermarket,captain morgan parrot bay coconut,rum,1.5,johnson


In [157]:
df.dtypes

Date            datetime64[ns]
Store Number            object
Store Type              object
product_name            object
category_new            object
Volume Sold            float64
County                  object
Month                    int64
dtype: object

# SPRING

In [161]:
rules_sp, frequent_itemsets_sp = mba(season=[3,4,5])

In [165]:
rules_sp

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(black velvet),(absolut swedish vodka 80 prf),0.903051,0.595187,0.559977,0.620095,1.041848,0.022493,1.065563
1,(absolut swedish vodka 80 prf),(black velvet),0.595187,0.903051,0.559977,0.940842,1.041848,0.022493,1.638822
2,(absolut swedish vodka 80 prf),(hawkeye vodka),0.595187,0.779315,0.510414,0.857569,1.100414,0.046576,1.549417
3,(hawkeye vodka),(absolut swedish vodka 80 prf),0.779315,0.595187,0.510414,0.654952,1.100414,0.046576,1.173207
4,(black velvet),(admiral nelson spiced rum),0.903051,0.613236,0.582782,0.645348,1.052364,0.028998,1.090544
5,(admiral nelson spiced rum),(black velvet),0.613236,0.903051,0.582782,0.950339,1.052364,0.028998,1.952202
6,(admiral nelson spiced rum),(hawkeye vodka),0.613236,0.779315,0.530153,0.864518,1.109330,0.052249,1.628881
7,(hawkeye vodka),(admiral nelson spiced rum),0.779315,0.613236,0.530153,0.680281,1.109330,0.052249,1.209699
8,(bacardi superior rum),(black velvet),0.575304,0.903051,0.545509,0.948210,1.050007,0.025980,1.871952
9,(black velvet),(bacardi superior rum),0.903051,0.575304,0.545509,0.604073,1.050007,0.025980,1.072663


In [162]:
frequent_itemsets_sp.sort_values(by='support', ascending=False).head(20)

,support,itemsets
3,0.903051,(black velvet)
8,0.779315,(hawkeye vodka)
26,0.725226,"(black velvet, hawkeye vodka)"
4,0.682739,(captain morgan spiced rum)
14,0.647185,(seagrams 7 crown bl whiskey)
23,0.644549,"(black velvet, captain morgan spiced rum)"
9,0.622891,(jack daniels old #7 black lbl)
7,0.622862,(five o'clock vodka)
11,0.619682,(jim beam)
1,0.613236,(admiral nelson spiced rum)


# SUMMER

In [172]:
rules_su, frequent_itemsets_su = mba(season=[6,7,8])
rules_su

Season


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(black velvet),(absolut swedish vodka 80 prf),0.891968,0.627283,0.586701,0.657760,1.048586,0.027184,1.089052
1,(absolut swedish vodka 80 prf),(black velvet),0.627283,0.891968,0.586701,0.935305,1.048586,0.027184,1.669860
2,(absolut swedish vodka 80 prf),(hawkeye vodka),0.627283,0.788966,0.539795,0.860529,1.090704,0.044890,1.513097
3,(hawkeye vodka),(absolut swedish vodka 80 prf),0.788966,0.627283,0.539795,0.684181,1.090704,0.044890,1.180157
4,(black velvet),(admiral nelson spiced rum),0.891968,0.615002,0.580012,0.650262,1.057332,0.031450,1.100817
5,(admiral nelson spiced rum),(black velvet),0.615002,0.891968,0.580012,0.943107,1.057332,0.031450,1.898849
6,(admiral nelson spiced rum),(hawkeye vodka),0.615002,0.788966,0.537378,0.873783,1.107504,0.052163,1.671995
7,(hawkeye vodka),(admiral nelson spiced rum),0.788966,0.615002,0.537378,0.681117,1.107504,0.052163,1.207334
8,(black velvet),(bacardi superior rum),0.891968,0.601962,0.566719,0.635358,1.055480,0.029789,1.091587
9,(bacardi superior rum),(black velvet),0.601962,0.891968,0.566719,0.941454,1.055480,0.029789,1.845248


In [173]:
frequent_itemsets_su.sort_values(by='support', ascending=False).head(20)

,support,itemsets
3,0.891968,(black velvet)
8,0.788966,(hawkeye vodka)
28,0.727812,"(black velvet, hawkeye vodka)"
4,0.678264,(captain morgan spiced rum)
14,0.643809,(seagrams 7 crown bl whiskey)
25,0.636052,"(black velvet, captain morgan spiced rum)"
9,0.635490,(jack daniels old #7 black lbl)
0,0.627283,(absolut swedish vodka 80 prf)
11,0.618346,(jim beam)
7,0.618262,(five o'clock vodka)


In [174]:
df.head()

,Date,Store Number,Store Type,product_name,category_new,Volume Sold,County,Month
0,2012-05-03,2636,Liquor Tobacco Store,nikolai vodka,vodka,21.0,polk,5
1,2012-07-05,3735,Liquor Tobacco Store,kessler blend whiskey,whisky,1.2,buchanan,7
2,2012-03-06,4631,Other,uv cake vodka,vodka,9.0,black hawk,3
3,2013-08-26,3162,Other Grocery or Convenience,windsor canadian pet,whisky,10.5,muscatine,8
4,2015-08-12,2513,Supermarket,captain morgan parrot bay coconut,rum,1.5,johnson,8


## NOTES

- Support: Its the default popularity of an item. In mathematical terms, the support of item A is nothing but the ratio of transactions involving A to the total number of transactions.

-- Support(Grapes) = (Transactions involving Grapes)/(Total transaction)

- Confidence: Likelihood that customer who bought both A and B. Its divides the number of transactions involving both A and B by the number of transactions involving B.

-- Confidence(A => B) = (Transactions involving both A and B)/(Transactions involving only A)

The support value for the first rule is 0.003. This number is calculated by dividing the number of transactions containing ‘avocado,’ ‘spaghetti,’ and ‘milk’ by the total number of transactions.

The confidence level for the rule is 0.416, which shows that out of all the transactions that contain both ‘avocado’ and ‘spaghetti’, 41.6 percent contain ‘milk’ too.

The lift of 1.241 tells us that ‘milk’ is 1.241 times more likely to be bought by the customers who buy both ‘avocado’ and ‘spaghetti’ compared to the default likelihood sale of ‘milk.’

# Downsampling Test

### Test sampling code

In [102]:
clean.loc[:,:] = clean.reset_index()

In [103]:
samp_size = int(1e5)

In [104]:
np.random.seed(7)
clean_rand = np.random.choice(clean.index, size=samp_size, replace=False)


In [105]:
clean_rand.sort()

In [106]:
clean_rand.max()

27372181

In [107]:
clean.shape

(27373000, 24)

In [108]:
clean.iloc[clean_rand]

,Date,Store Number,Vendor Number,Item Number,State Bottle Cost,State Bottle Retail,Bottles Sold,Sale (Dollars),Volume Sold (Liters),Store Name,Address,City,Zip Code,Store Location,County Number,County,Store Type,category_id,category_name,product_name,bottle_vol_ml,proof,category_new,imported
20,2015-02-12,3456,259.0,18120,14.97,22.46,1,22.46,0.75,quick shop / clear lake,904 n 8th st,clear lake,50428,POINT (-93.378772 43.142868),17.0,cerro gord,Convenience Store,1011250.0,single barrel bourbon whiskies,evan williams vintage,750.0,86.0,whisky,domestic
586,2012-10-30,4500,35.0,43126,7.53,11.30,6,67.80,4.50,casey's general store #2788 / north liberty,595 n kansas ave,north liberty,52317,POINT (-91.636952 41.751925),52.0,johnson,Convenience Store,1062200.0,puerto rico & virgin islands rum,bacardi superior rum,750.0,80.0,rum,domestic
839,2014-08-29,3612,434.0,41846,9.00,13.50,2,27.00,1.50,b and c liquor / maquoketa,509 e platt,maquoketa,52060,POINT (-90.659692 42.06922000000001),49.0,jackson,Liquor Tobacco Store,1101100.0,american alcohol,everclear alcohol,750.0,151.0,other,domestic
919,2012-02-02,2637,301.0,38176,9.64,14.46,4,57.84,3.00,hy-vee #5 / davenport,2351 w locust,davenport,52804,POINT (-90.611972 41.538207),NaN,NaN,Supermarket,1031100.0,american vodkas,titos handmade vodka,750.0,80.0,vodka,domestic
1279,2012-05-17,2579,115.0,11776,5.23,7.84,12,94.08,9.00,hy-vee food store / grinnell,320 west street south,grinnell,50112,POINT (-92.727343 41.7283),79.0,poweshiek,Supermarket,1012100.0,canadian whiskies,black velvet,750.0,80.0,whisky,domestic
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27371496,2012-10-02,2616,260.0,25606,6.53,9.79,12,117.48,9.00,hy-vee food and drug / clinton,901 south 4th st,clinton,52732,POINT (-90.193204 41.836268000000004),23.0,clinton,Drug Store,1011100.0,blended whiskies,seagrams 7 crown bl whiskey,750.0,80.0,whisky,domestic
27371524,2014-06-03,4239,260.0,10008,10.50,15.74,6,94.44,10.50,fareway stores #850 / spirit lake,1906 keokuk,spirit lake,51360,POINT (-95.106219 43.42157),30.0,dickinson,Supermarket,1012200.0,scotch whiskies,scoresby rare scotch,1750.0,80.0,whisky,domestic
27371718,2014-05-01,3565,259.0,35416,4.84,7.26,12,87.12,9.00,hartig drug store #10,701 mormon trek blvd,iowa city,52246,POINT (-91.574025 41.651984),52.0,johnson,Drug Store,1031080.0,vodka 80 proof,burnett's vodka 80 prf,750.0,80.0,vodka,domestic
27372036,2013-04-17,3013,205.0,52596,5.75,8.62,3,25.86,2.25,keith's foods,207 e locust st,bloomfield,52537,POINT (-92.412847 40.752691),26.0,davis,Other Grocery or Convenience,1051010.0,american grape brandies,e & j vs,750.0,80.0,brandy,domestic
